# Überblick: Wichtige Quantum Gates (Qiskit Basis)

Unten findest du eine kompakte Übersicht der wichtigsten elementaren Quanten-Gatter, die wir gleich verwenden oder erweitern können. Ein Qubit-Zustand lässt sich als |ψ⟩ = α|0⟩ + β|1⟩ schreiben (mit |α|²+|β|²=1). Gatter wirken linear (Unitary) auf diesen Vektor.

| Gate | Qiskit Befehl | Matrix / Wirkung (kurz) | Intuition |
|------|---------------|-------------------------|-----------|
| Identity (I) | `qc.id(q)` | [[1,0],[0,1]] | Tut nichts – Platzhalter / Timing |
| Pauli-X | `qc.x(q)` | [[0,1],[1,0]] | Bit-Flip: |0⟩ ↔ |1⟩ (klassisches NOT) |
| Pauli-Y | `qc.y(q)` | [[0,-i],[i,0]] | Kombination aus Flip + Phase (Rotation um Y-Achse) |
| Pauli-Z | `qc.z(q)` | [[1,0],[0,-1]] | Phasenflip: |1⟩ erhält ein − Zeichen |
| Hadamard (H) | `qc.h(q)` | (1/√2)[[1,1],[1,-1]] | Erzeugt Superposition: |0⟩→(|0⟩+|1⟩)/√2 |
| Phase (S) | `qc.s(q)` | diag(1, i) | Vierteldrehung um Z (π/2), fügt Phase zu |1⟩ hinzu |
| S† (Sdg) | `qc.sdg(q)` | diag(1, -i) | Inverse von S |
| T | `qc.t(q)` | diag(1, e^{iπ/4}) | Achteldrehung um Z (π/4) |
| T† (Tdg) | `qc.tdg(q)` | diag(1, e^{-iπ/4}) | Inverse von T |
| RX(θ) | `qc.rx(theta, q)` | Rotation um X | Feine kontrollierte Drehung |
| RY(θ) | `qc.ry(theta, q)` | Rotation um Y | Erzeugt kontrollierte Superposition |
| RZ(θ) | `qc.rz(theta, q)` | Rotation um Z | Ändert relative Phase |
| U(θ,φ,λ) | `qc.u(theta,phi,lmbd,q)` | Allgemeines 1-Qubit-Gate | Bel. Rotation (Euler-Zersetzung) |
| CX / CNOT | `qc.cx(c,t)` | Flipt Ziel falls Kontrolle=1 | Erzeugt Verschränkung |
| CZ | `qc.cz(c,t)` | Fügt Phase − zu |11⟩ hinzu | Phasenverschränkung |
| SWAP | `qc.swap(q1,q2)` | Vertauscht Zustände | Datenumlagerung |
| CCX (Toffoli) | `qc.ccx(c1,c2,t)` | Flipt Ziel falls beide Kontrollen=1 | Universell für klassische Logik |
| Barrier | `qc.barrier()` | — | Verhindert Compiler-Neuordnung |
| Measure | `qc.measure(q,c)` | Kollabiert Zustand → Bit | Liest Ergebnis (nicht unitär) |

Weitere nützliche:
- `qc.reset(q)` setzt ein Qubit zurück auf |0⟩ (nicht unitär, für Mid-Circuit Measurements nützlich)
- `qc.measure_all()` misst alle Qubits bequem.

Warum Superposition & Verschränkung? 
- Superposition (z.B. durch H) verteilt Amplituden über Basiszustände. 
- Verschränkung (z.B. H + CX) erzeugt Korrelationen, die klassisch nicht erklärt werden können.


> Tipp: Wenn du unsicher bist welches Gate du brauchst, schau zuerst hier rein oder nutze `qc.draw('mpl')` um den aktuellen Zustand der Schaltung zu visualisieren.


# Binder-Umgebung
Dieses Notebook läuft auf Binder. Eine lokale virtuelle Umgebung ist nicht erforderlich.

# Qiskit Workshop Demo 
Willkommen zur Qiskit Live-Demo! Imports für alle tests. Wichtig! Zu erst diese Zelle ausführen.

In [ ]:
# Abhängigkeiten sind bereits über requirements.txt installiert
# Falls du lokal ohne Installation arbeitest, entferne das Kommentarzeichen vor der nächsten Zeile:
# %pip install qiskit qiskit-aer matplotlib pylatexenc
import qiskit
import qiskit_aer
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram  # Für Histogramme weiter unten
import math  # Für Winkel / π etc.

# Gemeinsamer Simulator (kann in späteren Zellen direkt genutzt werden)
backend = Aer.get_backend('qasm_simulator')

print("Qiskit importiert (Basis-Imports + Simulator bereit)")

# Kapitel 0: Interaktive Bloch-Sphäre – Qubit Zustand fühlen
In diesem optionalen Einstiegs-Kapitel kannst du mit einem einzelnen Qubit spielen und sehen, wie sich Rotationen auf der Bloch-Sphäre auswirken.

Ziel:
- Verstehen: Ein Qubit-Zustand entspricht einem Punkt auf der Oberfläche der Bloch-Kugel.
- Rotationen um X / Y / Z verschieben diesen Punkt.
- Messwahrscheinlichkeit für 0/1 hängt von der Projektion auf die Z-Achse ab.

Was du tun kannst:
- Schiebe Slider für Winkel an (RX, RY, RZ) und beobachte den Effekt.
- Ändere Shots und sieh, wie sich das Histogramm stabilisiert.

Hinweis: Für die interaktive Darstellung nutzen wir `ipywidgets`. Falls die Slider nicht erscheinen: Stelle sicher, dass die Notebook-Umgebung Widgets unterstützt (Binder lädt meist automatisch die Extension).

In [ ]:
# Interaktive Bloch-Sphäre (optimiert)
# Schnell: analytische Berechnung ohne Simulator; optional: echter Simulator

from ipywidgets import FloatSlider, IntSlider, ToggleButtons, Button, HBox, VBox, Output
from qiskit.visualization import plot_bloch_vector, plot_histogram
import numpy as np, math

# Simulatoren (nur falls Modus = 'Simulator')
backend_state = Aer.get_backend('statevector_simulator')
backend_qasm = backend  # bereits vorhandener qasm_simulator

# --- Analytische Bloch-Vektor Berechnung für |0> --Rx--> --Ry--> --Rz-->
# Startzustand |0>: Bloch (0,0,1)

def bloch_after_rotations(rx, ry, rz):
    x, y, z = 0.0, 0.0, 1.0
    if abs(rx) > 1e-12:
        y, z = y*math.cos(rx) - z*math.sin(rx), y*math.sin(rx) + z*math.cos(rx)
    if abs(ry) > 1e-12:
        x, z = x*math.cos(ry) + z*math.sin(ry), -x*math.sin(ry) + z*math.cos(ry)
    if abs(rz) > 1e-12:
        x, y = x*math.cos(rz) - y*math.sin(rz), x*math.sin(rz) + y*math.cos(rz)
    return [x, y, z]

# Widgets (beschreibende Labels für Einsteiger)
slider_rx = FloatSlider(min=0, max=2*math.pi, step=0.05, value=0.0, description='X-Drehung')  # kippt zwischen |0> und |1>
slider_ry = FloatSlider(min=0, max=2*math.pi, step=0.05, value=0.0, description='Y-Drehung')  # mischt Wahrsch.
slider_rz = FloatSlider(min=0, max=2*math.pi, step=0.05, value=0.0, description='Z-Phase')    # ändert Phase
slider_shots = IntSlider(min=50, max=2000, step=50, value=500, description='Messungen')
mode = ToggleButtons(options=['Schnell','Simulator'], description='Modus')
btn_update = Button(description='Aktualisieren', button_style='primary', tooltip='Manuell neu rendern')
output = Output()

help_text = Output()
with help_text:
    print("Erklärung der Regler:")
    print("- X-Drehung (Rx): Rotation um X-Achse. Bewegt Zustand von Nordpol Richtung Südpol (ändert Mess-Wahrsch.).")
    print("- Y-Drehung (Ry): Rotation um Y-Achse. Ebenfalls Änderung der 0/1-Wahrscheinlichkeiten.")
    print("- Z-Phase (Rz): Drehung um Z-Achse. Ändert NUR die Phase (sichtbar erst nach Basiswechsel).")
    print("- Messungen: Anzahl der Wiederholungen zur statistischen Approximation.")
    print("- Modus 'Schnell': Formeln ohne echten Simulator. 'Simulator': echte Statevector + Messungen.")

rendering = False

def render():
    global rendering
    if rendering:  # einfache Reentrancy-Guard
        return
    rendering = True
    with output:
        output.clear_output(wait=True)
        rx, ry, rz, shots = slider_rx.value, slider_ry.value, slider_rz.value, slider_shots.value
        if mode.value == 'Schnell':
            bloch = bloch_after_rotations(rx, ry, rz)
            z = bloch[2]
            p0 = (1+z)/2
            p1 = 1-p0
            c0 = int(round(p0*shots))
            c1 = shots - c0
            counts = {'0': c0, '1': c1}
            print(f"Analytisch | Rx={rx:.2f} Ry={ry:.2f} Rz={rz:.2f} -> p(0)≈{p0:.3f} p(1)≈{p1:.3f}")
            qc = QuantumCircuit(1,1)
            if rx: qc.rx(rx,0)
            if ry: qc.ry(ry,0)
            if rz: qc.rz(rz,0)
            try:
                display(qc.draw('mpl'))
            except Exception:
                print(qc.draw())
            display(plot_bloch_vector(bloch, title='Bloch-Sphäre (analytisch)'))
            display(plot_histogram(counts, title='Messung (approximiert)'))
        else:
            qc = QuantumCircuit(1,1)
            if rx: qc.rx(rx,0)
            if ry: qc.ry(ry,0)
            if rz: qc.rz(rz,0)
            sv = backend_state.run(qc).result().get_statevector(qc)
            a, b = sv[0], sv[1]
            x = 2 * np.real(np.conjugate(a) * b)
            y = 2 * np.imag(np.conjugate(a) * b)
            z = np.abs(a)**2 - np.abs(b)**2
            bloch = [x,y,z]
            qc_m = qc.copy(); qc_m.measure(0,0)
            counts = backend_qasm.run(qc_m, shots=shots).result().get_counts(qc_m)
            print(f"Simulator | Rx={rx:.2f} Ry={ry:.2f} Rz={rz:.2f} Counts={counts}")
            try:
                display(qc.draw('mpl'))
            except Exception:
                print(qc.draw())
            display(plot_bloch_vector(bloch, title='Bloch-Sphäre (Simulator)'))
            display(plot_histogram(counts, title='Messung (Simulator)'))
    rendering = False

# Auto-Update bei Slider-Änderungen (debounce light via manual button optional)
for w in (slider_rx, slider_ry, slider_rz, slider_shots, mode):
    w.observe(lambda change: render() if change['name']=='value' else None, names='value')
btn_update.on_click(lambda _: render())

controls = VBox([
    help_text,
    HBox([slider_rx, slider_ry, slider_rz]),
    HBox([slider_shots, mode, btn_update])
])

display(controls)
display(output)  # wichtig: Output anzeigen
render()  # erste Ausgabe

## Kapitel 1: Einfacher Einzel-Qubit-Circuit – Wie oft messen wir 0 oder 1?
Wir erstellen einen Circuit mit 1 Qubit und 1 klassischem Bit. Ein Hadamard-Gatter `H` bringt das Qubit in eine Superposition, sodass bei vielen Messungen ungefähr gleich oft `0` und `1` auftreten. Wir simulieren direkt und schauen uns die Häufigkeiten an.

In [ ]:
# Einfaches Beispiel: 1 Qubit, Hadamard, Messung
# Ziel: Zeigen, dass ein einzelnes Qubit nach einem Hadamard ungefähr zu 50% als 0 und 50% als 1 gemessen wird.

# Anzahl der wiederholungen von dem Circuit.
shots = 100

# Erzeuge einen Circuit mit 1 Qubit und 1 klassischem Bit. Klassisches Bit wird benutzt um das Ergebnis zu speichern
qc_simple = QuantumCircuit(1,1) # Qubit,Bit

# Hadamard-Gatter auf Qubit 0: |0> -> (|0> + |1>)/√2 (Superposition)
qc_simple.h(0)

# Messe Qubit 0 in das klassische Bit 0 (kollabiert Superposition zu 0 oder 1)
qc_simple.measure(0,0)

# Zeichne die Schaltung (mit 'mpl')
try:
    display(qc_simple.draw('mpl'))
except Exception:
    print(qc_simple.draw())

# Simuliere die Schaltung: 'qasm_simulator' führt viele Messwiederholungen (shots) aus
result = backend.run(qc_simple, shots=shots).result()  # 100 Wiederholungen -> statistische Verteilung

# Hole die Zählungen der gemessenen Bitstrings (hier nur '0' oder '1')
counts_simple = result.get_counts(qc_simple)
print("Häufigkeiten (Anzahl der Messungen pro Ergebnis):", counts_simple)

# Plot als Histogramm zur schnellen visuellen Einschätzung (sollte ~gleich hoch sein)
plot_histogram(counts_simple)

## Kapitel 2: Wie erzwinge oder beeinflusse ich Messergebnisse?
Wir schauen uns drei kleine Varianten an (alles noch 1 Qubit):

1. Immer 1 messen: Zustandsvorbereitung durch ein X-Gatter (NOT) vor der Messung.
2. Wahrscheinlichkeiten steuern: Rotation `ry(θ)` – verändert Anteil für 0/1 kontinuierlich.
3. Phase ohne Mess-Effekt (hier): Z-Phase ändert nur relative Phase, nicht die Messwahrscheinlichkeit in Z-Basis.

> Merke: Nur Amplitudenbeträge (|α|², |β|²) bestimmen Messwahrscheinlichkeiten in der Computational Basis. Reine Phase (global oder passend lokal) kann unsichtbar bleiben, bis wir in eine andere Basis rotieren.

### Variante 1: Immer 1 erzwingen mit X-Gate
Wir setzen das Qubit per X-Gatter direkt in den Zustand |1>, damit die Messung (fast) immer 1 liefert.

In [ ]:
# Variante 1: Immer 1 messen (X)
# Ziel: Zeigen, wie wir das Ergebnis fest auf 1 setzen.

shots = 500  # Anzahl Wiederholungen

qc_one = QuantumCircuit(1,1)
qc_one.x(0)          # Macht aus |0> das |1>
qc_one.measure(0,0)  # Messergebnis ins klassische Bit

counts_one = backend.run(qc_one, shots=shots).result().get_counts(qc_one)
print("Variante 1 - Immer 1:", counts_one)
plot_histogram(counts_one, title='Variante 1: X -> immer 1')

### Variante 2: Wahrscheinlichkeiten fein einstellen mit RY(θ)
Mit einer Rotation um die Y-Achse stellen wir kontinuierlich ein, wie oft 0 oder 1 fällt.

In [ ]:
# Variante 2: Wahrscheinlichkeiten steuern mit RY(θ)
# Ziel: θ bestimmt wie oft 1 herauskommt (glatte Einstellmöglichkeit)

shots = 500
θ = math.pi/3          # Beispielwinkel ~60°

qc_bias = QuantumCircuit(1,1)
qc_bias.ry(θ,0)        # Drehung erzeugt Mischung zwischen 0 und 1
qc_bias.measure(0,0)

counts_bias = backend.run(qc_bias, shots=shots).result().get_counts(qc_bias)
print(f"Variante 2 - RY({θ:.2f}):", counts_bias)
plot_histogram(counts_bias, title=f'Variante 2: RY({θ:.2f}) Bias')

### Variante 3: Phase sichtbar machen über Basiswechsel (H–Z–H)
Die Z-Phase allein ändert die Messwahrscheinlichkeit nicht. Mit H davor und danach wird sie zu einem effektiven Flip.

In [ ]:
# Variante 3: Phase + Basiswechsel (H-Z-H)
# Zeigt: Eine Phase (Z) allein ändert Messwahrscheinlichkeit nicht –
# aber durch Basiswechsel (H davor und danach) kann sie wie ein Flip wirken.

shots = 500

qc_phase = QuantumCircuit(1,1)
qc_phase.h(0)      # Wechsel in Superposition
qc_phase.z(0)      # Phase auf den |1>-Anteil
qc_phase.h(0)      # Zurück -> Phase wird hier zu einem Bit-Flip-Effekt
qc_phase.measure(0,0)

counts_phase = backend.run(qc_phase, shots=shots).result().get_counts(qc_phase)
print("Variante 3 - H-Z-H:", counts_phase)
plot_histogram(counts_phase, title='Variante 3: H-Z-H')

## Kapitel 3: CNOT (CX) ganz einfach erklärt 2 QUBITS
Das CNOT (CX) hat zwei Qubits:
- Oben: Kontrolle ("Control").
- Unten: Ziel ("Target").

Regel (Merksatz): Ist die Kontrolle = 1, dann mache ein X (Flip) auf dem Ziel. Ist die Kontrolle = 0, tue nichts.

Man kann es wie einen "bedingten Lichtschalter" sehen: Control = 1 schaltet den Zustand des Ziel-Qubits um.

### Wahrheitstabelle (eingängig)
| Control | Target (vorher) | Target (nachher) | Ausgabe (Control Target) | Was passiert? |
|---------|-----------------|------------------|---------------------------|---------------|
| 0       | 0               | 0                | 00                        | Nichts        |
| 0       | 1               | 1                | 01                        | Nichts        |
| 1       | 0               | 1                | 11                        | Flip          |
| 1       | 1               | 0                | 10                        | Flip          |

Kurz: Nur in den unteren beiden Zeilen (Control=1) wechselt das Ziel.

### Warum ist das wichtig?
- So bauen wir Verschränkung: z.B. erst Hadamard auf das Control (Superposition), dann CX → erzeugt Bell-Zustände.
- So können wir Logik (IF) in Quanten-Schaltungen nachbilden.

Gleich unten erstellen wir jede der vier Kombinationen und prüfen, was rauskommt.


In [ ]:
# CNOT Wahrheitstabelle per Simulation
# Wir erzeugen die vier Basiszustände |00>, |01>, |10>, |11>, wenden CX an und messen.


shots = 256
kombinationen = {
    "00": [],
    "01": [(1, 'x')],        # Ziel (unten) auf 1 setzen
    "10": [(0, 'x')],        # Control (oben) auf 1 setzen
    "11": [(0, 'x'), (1, 'x')]  # Beide auf 1 setzen
}

results = {}

for name, ops in kombinationen.items():
    qc = QuantumCircuit(2,2)
    # Vorbereitung
    for qubit, art in ops:
        if art == 'x':
            qc.x(qubit)
    # CX: Kontrolle = Qubit 0, Ziel = Qubit 1
    qc.cx(0,1)
    qc.measure([0,1],[0,1])

    # Circuit zeichnen (so sieht das Gatter in dieser Eingabe aus)
    print(f"\nCircuit für Eingang {name}:")
    try:
        display(qc.draw('mpl'))
    except Exception:
        print(qc.draw())

    counts = backend.run(qc, shots=shots).result().get_counts(qc)
    results[name] = counts
    print(f"Eingang {name} -> Ausgabe Counts: {counts}")

# Zusammenfassung in einer kleinen Tabelle drucken
print("\nZusammenfassung (dominierender Output):")
for eingang, counts in results.items():
    # größten Key finden
    dominant = max(counts.items(), key=lambda kv: kv[1])[0]
    print(f"{eingang} -> {dominant}")